<a href="https://colab.research.google.com/github/darkstarlegend/mlproj_rainfall/blob/main/mlproj_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,RobustScaler
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor,XGBClassifier

In [2]:
from sklearn.metrics  import r2_score

In [3]:
!pip install XGBoost


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
df=pd.read_csv('/content/weatherAUS.csv')

In [5]:
df.head(5)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [7]:
df.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,143975.000000,144199.000000,142199.000000,82670.000000,75625.000000,135197.000000,143693.000000,142398.000000,142806.000000,140953.000000,130395.00000,130432.000000,89572.000000,86102.000000,143693.000000,141851.00000
mean,12.194034,23.221348,2.360918,5.468232,7.611178,40.035230,14.043426,18.662657,68.880831,51.539116,1017.64994,1015.255889,4.447461,4.509930,16.990631,21.68339
std,6.398495,7.119049,8.478060,4.193704,3.785483,13.607062,8.915375,8.809800,19.029164,20.795902,7.10653,7.037414,2.887159,2.720357,6.488753,6.93665
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.50000,977.100000,0.000000,0.000000,-7.200000,-5.40000
25%,7.600000,17.900000,0.000000,2.600000,4.800000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.90000,1010.400000,1.000000,2.000000,12.300000,16.60000
50%,12.000000,22.600000,0.000000,4.800000,8.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.60000,1015.200000,5.000000,5.000000,16.700000,21.10000
75%,16.900000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.40000,1020.000000,7.000000,7.000000,21.600000,26.40000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.00000,1039.600000,9.000000,9.000000,40.200000,46.70000


In [8]:
df.isnull().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [9]:
df.dtypes

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
dtype: object

In [10]:
df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

In [11]:
numerical_feature =[feature for feature in df.columns if df[feature].dtypes != 'O']
descrete_feature=[feature for feature in numerical_feature if len(df[feature].unique())<25 ]
continous_feature=[feature for feature in numerical_feature if feature not in descrete_feature ]
categorical_feature =[feature for feature in df.columns if feature not in numerical_feature]

In [12]:
descrete_feature

['Cloud9am', 'Cloud3pm']

In [13]:
df.isnull().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [14]:
def random_sample_imp(df,variable):
  random_sample=df[variable].dropna().sample(df[variable].isnull().sum(),random_state=42)
  random_sample.index=df[df[variable].isnull()].index
  df.loc[df[variable].isnull(),variable]=random_sample


In [15]:
"""
random_sample_imp(df,'Cloud3pm')
random_sample_imp(df,'Cloud9am')
random_sample_imp(df,'Evaporation')
random_sample_imp(df,'Sunshine')
"""

"\nrandom_sample_imp(df,'Cloud3pm')\nrandom_sample_imp(df,'Cloud9am')\nrandom_sample_imp(df,'Evaporation')\nrandom_sample_imp(df,'Sunshine')\n"

In [16]:
df.drop('Date',axis=1,inplace=True)

In [17]:
"""
for feature in continous_feature:
  data=df.copy()
  sns.histplot(df[feature])
  plt.xlabel(feature)
  plt.ylabel('count')
  plt.title(feature)
  plt.figure(figsize=(15,15))
  plt.show()
"""

"\nfor feature in continous_feature:\n  data=df.copy()\n  sns.histplot(df[feature])\n  plt.xlabel(feature)\n  plt.ylabel('count')\n  plt.title(feature)\n  plt.figure(figsize=(15,15))\n  plt.show()\n"

In [18]:
"""
for feature in continous_feature:
  data=df.copy()
  sns.boxplot(df[feature])
  plt.xlabel(feature)
  plt.ylabel('count')
  plt.title(feature)
  plt.figure(figsize=(15,15))
  plt.show()
"""

"\nfor feature in continous_feature:\n  data=df.copy()\n  sns.boxplot(df[feature])\n  plt.xlabel(feature)\n  plt.ylabel('count')\n  plt.title(feature)\n  plt.figure(figsize=(15,15))\n  plt.show()\n"

In [19]:
"""
for feature in continous_feature:
  df[feature].fillna(df[feature].median(),inplace=True)
"""

'\nfor feature in continous_feature:\n  df[feature].fillna(df[feature].median(),inplace=True)\n'

In [20]:
df['RainToday'].unique()

array(['No', 'Yes', nan], dtype=object)

In [21]:
df['RainToday']=df['RainToday'].fillna('No')

In [22]:
df['RainTomorrow'].unique()

array(['No', 'Yes', nan], dtype=object)

In [23]:
df['RainTomorrow']=df['RainTomorrow'].fillna('No')

In [24]:
encoder =LabelEncoder()
label_encoder_columns =['RainToday','RainTomorrow']

for column in label_encoder_columns:
  df[column]=encoder.fit_transform(df[column])

In [25]:
def add_column_prefix(data,column,prefix):
  return data[column].apply(lambda x: prefix + str(x))

In [26]:
add_column_prefix(df,'Location','loc_')

0         loc_Albury
1         loc_Albury
2         loc_Albury
3         loc_Albury
4         loc_Albury
             ...    
145455     loc_Uluru
145456     loc_Uluru
145457     loc_Uluru
145458     loc_Uluru
145459     loc_Uluru
Name: Location, Length: 145460, dtype: object

In [27]:
df['Humidity3pm'].unique()

array([ 22.,  25.,  30.,  16.,  33.,  23.,  19.,   9.,  27.,  91.,  93.,
        43.,  32.,  28.,  82.,  65.,  26.,  24.,  17.,  15.,  70.,  12.,
         8.,  31.,  20.,  10.,  21.,  39.,  13.,  11.,  69.,  18.,  14.,
        35.,  90.,  68.,  74.,  41.,  34.,  78.,  49.,  37.,  42.,  47.,
        52.,  29.,  86.,  89.,  62.,  79.,  48.,  46.,  38.,  44.,  51.,
        36.,  45.,  40.,  53.,  61.,  59.,  63.,  54.,  57.,  50.,  58.,
        75.,  97.,  80.,  95.,  56.,  72.,  60.,  71.,  73.,  81.,  76.,
        64.,  87.,  85.,  67.,  55.,  77.,  84.,   7.,  99.,  92.,  66.,
        83.,  94.,  96.,  88.,  nan, 100.,  98.,   5.,   6.,   1.,   4.,
         2.,   3.,   0.])

In [28]:
df['WindDir9am']=add_column_prefix(df,'WindDir9am','9_')
df['WindDir3pm']=add_column_prefix(df,'WindDir3pm','3_')

In [29]:
pd.get_dummies(df['WindGustDir'])

,E,ENE,ESE,N,NE,NNE,NNW,NW,S,SE,SSE,SSW,SW,W,WNW,WSW
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
145456,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
145457,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
145458,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [30]:
def onehot_encoder(data,columns):
  for column in columns:
    dummies=pd.get_dummies(data[column])
    data=pd.concat([data,dummies],axis=1)
    data.drop(column,axis=1,inplace=True)
  return data

In [31]:
categorical_features =['Location','WindGustDir','WindDir9am','WindDir3pm']
df=onehot_encoder(df,categorical_features)

In [32]:
df.dtypes

MinTemp        float64
MaxTemp        float64
Rainfall       float64
Evaporation    float64
Sunshine       float64
                ...   
3_SW             uint8
3_W              uint8
3_WNW            uint8
3_WSW            uint8
3_nan            uint8
Length: 117, dtype: object

In [33]:
def impute_mean(data,columns):
  for column in columns:
    data[column]=data[column].fillna(data[column].mean())

In [34]:
na_columns=[ 
'MinTemp',        
'MaxTemp',          
'Rainfall',     
'Evaporation',  
'Sunshine',        
'WindGustSpeed', 
'WindSpeed9am',    
'WindSpeed3pm', 
'Cloud9am',
'Cloud3pm',
'Temp9am',
'Temp3pm'  
]

In [35]:
impute_mean(df,na_columns)

In [36]:
df.isnull().sum()

MinTemp        0
MaxTemp        0
Rainfall       0
Evaporation    0
Sunshine       0
              ..
3_SW           0
3_W            0
3_WNW          0
3_WSW          0
3_nan          0
Length: 117, dtype: int64

In [47]:
df.dropna(inplace=True)

In [48]:
y=df['RainTomorrow']
X=df.drop('RainTomorrow',axis=1)

In [49]:
scaler=RobustScaler()
X=pd.DataFrame(scaler.fit_transform(X),columns=X.columns)


In [39]:
X

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,3_NW,3_S,3_SE,3_SSE,3_SSW,3_SW,3_W,3_WNW,3_WSW,3_nan
0,0.142857,0.019608,0.6,0.0,0.0,0.333333,0.583333,0.485213,0.038462,-1.034483,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-0.516484,0.235294,0.0,0.0,0.0,0.333333,-0.750000,0.303395,-1.000000,-0.931034,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.087912,0.294118,0.0,0.0,0.0,0.466667,0.500000,0.667031,-1.230769,-0.758621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.318681,0.519608,0.0,0.0,0.0,-1.000000,-0.166667,-0.878423,-0.961538,-1.241379,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.593407,0.941176,1.0,0.0,0.0,0.133333,-0.500000,0.121577,0.461538,-0.655172,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,-1.021978,0.068627,0.0,0.0,0.0,-0.533333,0.000000,-0.696605,-0.730769,-0.965517,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
145456,-0.934066,0.254902,0.0,0.0,0.0,-1.133333,0.000000,-0.878423,-0.538462,-1.068966,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
145457,-0.736264,0.411765,0.0,0.0,0.0,-0.133333,-0.333333,-0.878423,-0.653846,-0.965517,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
145458,-0.472527,0.421569,0.0,0.0,0.0,-0.733333,0.000000,-1.060242,-0.730769,-0.965517,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8,random_state=42)

In [41]:
xgr = XGBClassifier()
xgr.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [42]:
y_pred2 = xgr.predict(X_test)
print("XGBoost - R2: ", r2_score(y_test, y_pred2))

XGBoost - R2:  0.17053738867380308


In [58]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier


In [61]:
import scikitplot as skplt


In [51]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)

RandomForestClassifier()

In [52]:
y_pred1 = rf.predict(X_test)
print(confusion_matrix(y_test,y_pred1))
print(accuracy_score(y_test,y_pred1))
print(classification_report(y_test,y_pred1))

[[19099   800]
 [ 2799  2820]]
0.8589623011207775
              precision    recall  f1-score   support

           0       0.87      0.96      0.91     19899
           1       0.78      0.50      0.61      5619

    accuracy                           0.86     25518
   macro avg       0.83      0.73      0.76     25518
weighted avg       0.85      0.86      0.85     25518



In [64]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [65]:
y_pred2 = logreg.predict(X_test)
print(confusion_matrix(y_test,y_pred2))
print(accuracy_score(y_test,y_pred2))
print(classification_report(y_test,y_pred2))

[[18913   986]
 [ 2698  2921]]
0.8556313190688926
              precision    recall  f1-score   support

           0       0.88      0.95      0.91     19899
           1       0.75      0.52      0.61      5619

    accuracy                           0.86     25518
   macro avg       0.81      0.74      0.76     25518
weighted avg       0.85      0.86      0.85     25518



In [67]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [68]:
y_pred3 = gnb.predict(X_test)
print(confusion_matrix(y_test,y_pred3))
print(accuracy_score(y_test,y_pred3))
print(classification_report(y_test,y_pred3))

[[12822  7077]
 [ 1629  3990]]
0.658829061838702
              precision    recall  f1-score   support

           0       0.89      0.64      0.75     19899
           1       0.36      0.71      0.48      5619

    accuracy                           0.66     25518
   macro avg       0.62      0.68      0.61     25518
weighted avg       0.77      0.66      0.69     25518



In [69]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [70]:
y_pred4 = knn.predict(X_test)
print(confusion_matrix(y_test,y_pred4))
print(accuracy_score(y_test,y_pred4))
print(classification_report(y_test,y_pred4))

[[18329  1570]
 [ 2884  2735]]
0.825456540481229
              precision    recall  f1-score   support

           0       0.86      0.92      0.89     19899
           1       0.64      0.49      0.55      5619

    accuracy                           0.83     25518
   macro avg       0.75      0.70      0.72     25518
weighted avg       0.81      0.83      0.82     25518



In [71]:
svc = SVC()

svc.fit(X_train, y_train)
y_pred5 = svc.predict(X_test)
print(confusion_matrix(y_test,y_pred5))
print(accuracy_score(y_test,y_pred5))
print(classification_report(y_test,y_pred5))

[[19184   715]
 [ 3006  2613]]
0.854181362175719
              precision    recall  f1-score   support

           0       0.86      0.96      0.91     19899
           1       0.79      0.47      0.58      5619

    accuracy                           0.85     25518
   macro avg       0.82      0.71      0.75     25518
weighted avg       0.85      0.85      0.84     25518



In [72]:
import joblib
joblib.dump(rf, "rf.pkl")
#joblib.dump(cat, "cat.pkl")
joblib.dump(logreg, "logreg.pkl")
joblib.dump(gnb, "gnb.pkl")
joblib.dump(knn, "knn.pkl")
joblib.dump(svc, "svc.pkl")
#joblib.dump(xgb, "xgb.pkl")

['svc.pkl']